# Rely on LLM's to extract info from docs

Will save them in JSON for future reuse.

In [1]:
import os
import boto3
import json
import dateparser
from glob import glob
# from rag.basic_retrieval import file_id
# from cachier import cachier
from typing import List


from Templates.aws_markdown_template import TEMPLATE as MARKDOWN_TEMPLATE
from Templates.aws_templates_common import build_aws_template

from IPython.display import Markdown

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# PDF_LOCATION = 'IndustrySource/Misc/3D Printer Manufacturing in the US.pdf'
PDF_LOCATION = 'IndustrySource/Misc/HVAC%20Service%20Franchises%20in%20the%20US.pdf'
DOC_ID = PDF_LOCATION.split('/')[-1].split('.')[0].lower().replace(' ', '-')
OUTPUT_FOLDER = f'./rag_outputs/{DOC_ID}'
if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

In [3]:
# AWS_REGION_NAME = 'us-west-2'
AWS_REGION_NAME = 'us-east-1'

aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')

# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime.html
bedrock = boto3.client(
    service_name='bedrock-runtime',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=AWS_REGION_NAME
)

In [27]:
from pypdf import PdfReader
import time

def get_number_of_pages(filename: str) -> int:
    with open(filename, 'rb') as f:
        inputpdf = PdfReader(f)
        return len(inputpdf.pages)

In [4]:
MARKDOWN_PROMPT = """
We are sequentially converting a pdf document page by page to markdown format. You are an expert in converting the given PDF content to a Markdown representation.

Follow these instructions to complete the task:
- Infer the headings and subheadings of the given content with their levels from the appearance and semantic context. Generally, the larger font size, more visible color, and boldface indicate a lower level of the heading. For example, level 1 headings are expected to be more prominent than level 2 headings.
- Depending on the level of the section, you use an appropriate number of hash signs (#) to mark their headers in markdown format. # for level 1, ## for level 2, ### for level 3 and so on.
- The provided content may start from any page of the document. So the heading at first does not necessarily mean a level 1 heading.
- Do not insert any new content. Just convert the existing content to Markdown format while keeping the structure.
- Text is converted as it is. If the text is present in conflicting format that can be confusing, please interpret the text correctly and convert it to markdown.
- For visuals such as graphs, plots and figures, interpret them, be objective and explain the interpretation of data in detail with numbers and use that explanation in place of the visuals. Explanation provided should capture all the data insights that can be inferred from the figure. Use your best judgement to interpret the visuals.
- For tables, convert them to markdown table format without any explanation.
- The output should be in markdown format. Do not modify any content. 
Next is the pdf content:\n\n
"""

In [5]:
def get_raw_pdf_part(filename: str) -> dict:
    """This works best and parses quickly."""
    with open(filename, 'rb') as f:
        content = f.read()
        return {
            "document": {
                "format": "pdf",
                "name": 'document',
                "source": {
                    "bytes": content
                }
            }
        }


def response_to_template(filename: str, template: dict, prompt: str) -> dict:
    initial_message = {
        "role": "user",
        "content": [
            {
                "text": prompt,
            },
        ],
    }

    initial_message['content'].append(get_raw_pdf_part(filename))
    

    tool_list = [{
        "toolSpec": template
    }]
    model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
    response = bedrock.converse(
        modelId=model_id,
        # modelId="meta.llama3-1-405b-instruct-v1:0",
        messages=[initial_message],
        inferenceConfig={
            "temperature": 0
        },
        toolConfig={
            "tools": tool_list,
            "toolChoice": {
                "tool": {
                    "name": "info_extract"
                }
            }
        }
    )
    core_response = response['output']['message']['content'][0]['toolUse']['input']
    if 'properties' in core_response:
        core_response: dict = core_response['properties']
    for k, v in core_response.items():
        if isinstance(v, str) and v[0] in '{[' and v[-1] in ']}':
            try:
                core_response[k] = json.loads(v)
            except Exception:
                pass

    return core_response

In [6]:

# @cachier(hash_func=filename_template_hash)
def info_from_doc_template(filename: str, template: dict, prompt: str, **kwargs) -> dict:
    """Populate the separate templates and merge the result."""

    template_parts = template['data']
    full_templates = build_aws_template(template_parts)
    results = [response_to_template(filename, part, prompt, **kwargs) for part in full_templates]

    total = {}
    for result in results:
        total.update(result)

    return total

In [7]:
def extract_markdown(filename: str, first_page: int, last_page: int) -> dict:
    pages_filename = get_initial_pages(filename, pmin=first_page, pmax=last_page+1)
    result = info_from_doc_template(filename=pages_filename, template=MARKDOWN_TEMPLATE, prompt=MARKDOWN_PROMPT)
    os.remove(pages_filename)
    return result

In [8]:
def sequentially_process_pdf(filename, np=1):
    # find total number of pages in the pdf document
    total_pages = get_number_of_pages(filename)

    results = []
    skipped_pages = []
    for i in range(0, total_pages, np):
        print(f"Processing pages {i} to {i+np}")
        pages_filename = get_initial_pages(filename, pmin=i, pmax=i+np)
        failures = 0
        success = False
        result = None
        while (not success) and (failures < 3):
            try:
                result = info_from_doc_template(filename=pages_filename, template=MARKDOWN_TEMPLATE, prompt=MARKDOWN_PROMPT)
                success = True
                time.sleep(5)
            except:
                print(f"Error processing page {i+1}")
                failures += 1
                print(f"Retrying in 60 seconds.")
                time.sleep(60)
                if failures == 3:
                    result = {
                        'markdown': "**skipped**",
                    }
                    print(f"Failed to process page {i+1} after 3 attempts.")
                    skipped_pages.append(i+1)

        results.append(result)
        os.remove(pages_filename)
    print(f" Had to skip pages: {len(skipped_pages)}")
    return results, skipped_pages

        

    

In [9]:
file_path = PDF_LOCATION

results, skipped_pages = sequentially_process_pdf(file_path, np=1)

Processing pages 0 to 1
Processing pages 1 to 2
Error processing page 2
Retrying in 60 seconds.
Processing pages 2 to 3
Error processing page 3
Retrying in 60 seconds.
Processing pages 3 to 4
Error processing page 4
Retrying in 60 seconds.
Processing pages 4 to 5
Error processing page 5
Retrying in 60 seconds.
Processing pages 5 to 6
Error processing page 6
Retrying in 60 seconds.
Processing pages 6 to 7
Error processing page 7
Retrying in 60 seconds.
Processing pages 7 to 8
Error processing page 8
Retrying in 60 seconds.
Processing pages 8 to 9
Error processing page 9
Retrying in 60 seconds.
Processing pages 9 to 10
Error processing page 10
Retrying in 60 seconds.
Processing pages 10 to 11
Error processing page 11
Retrying in 60 seconds.
Processing pages 11 to 12
Error processing page 12
Retrying in 60 seconds.
Processing pages 12 to 13
Error processing page 13
Retrying in 60 seconds.
Processing pages 13 to 14
Error processing page 14
Retrying in 60 seconds.
Processing pages 14 to 15


In [11]:
print(len(results))

53


In [12]:
from llama_index.core import Document
import pickle
model = "anthropic.claude-3-5-sonnet-20240620-v1:0"

output_pickle_path = os.path.join(OUTPUT_FOLDER, f"_{model}_pages.pkl")

documents = []
for i, response in enumerate(results):
    doc = Document(text=results[i]['markdown'], metadata={"page_number": i+1})
    documents.append(doc)
parsed_result = {"parsed_documents": documents, "skipped_pages": skipped_pages}
pickle.dump(parsed_result, open(output_pickle_path, 'wb'))






In [ ]:
# filenames = [
#     'IndustrySource/Misc/3D Printer Manufacturing in the US.pdf'
# ]
# markdown = [extract_markdown(filename, 45, 48) for filename in filenames[:5]]
# markdown

# Phase 2 : Load documents and do post processing to correct the heading structure

In [4]:
from llama_index.core import Document
import pickle

# PDF_LOCATION = 'IndustrySource/Misc/3D Printer Manufacturing in the US.pdf'
PDF_LOCATION = 'IndustrySource/Misc/HVAC%20Service%20Franchises%20in%20the%20US.pdf'
DOC_ID = PDF_LOCATION.split('/')[-1].split('.')[0].lower().replace(' ', '-')
OUTPUT_FOLDER = f'./rag_outputs/{DOC_ID}'


model = "anthropic.claude-3-5-sonnet-20240620-v1:0"

output_pickle_path = os.path.join(OUTPUT_FOLDER, f"_{model}_pages.pkl")
print(output_pickle_path)
# load pickle file if it exists
if os.path.exists(output_pickle_path):
    with open(output_pickle_path, 'rb') as f:
        parsed_result = pickle.load(f)

documents = parsed_result['parsed_documents']
print(len(documents))

./rag_outputs/hvac%20service%20franchises%20in%20the%20us/_anthropic.claude-3-5-sonnet-20240620-v1:0_pages.pkl
53


In [5]:
def response_to_text(content_text: str, template: dict, main_prompt: str, system_prompt: str, final_prompt: str, tool_name: str="info_extract") -> dict:
    initial_message = {
        "role": "user",
        "content": [
            {
                "text": main_prompt,
            },
        ],
    }

    initial_message['content'].append({"text": content_text})
    if final_prompt is not None:
        initial_message['content'].append({"text": final_prompt})
    

    tool_list = [{
        "toolSpec": template
    }]
    # model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
    model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"
    response = bedrock.converse(
        modelId=model_id,
        # modelId="meta.llama3-1-405b-instruct-v1:0",
        messages=[initial_message],
        # system = system_prompt,
        inferenceConfig={
            "temperature": 0,
        },
        toolConfig={
            "tools": tool_list,
            "toolChoice": {
                "tool": {
                    "name": tool_name
                }
            }
        }
    )
    core_response = response['output']['message']['content'][0]['toolUse']['input']
    if 'properties' in core_response:
        core_response: dict = core_response['properties']
    for k, v in core_response.items():
        if isinstance(v, str) and v[0] in '{[' and v[-1] in ']}':
            try:
                core_response[k] = json.loads(v)
            except Exception:
                pass

    return core_response, response

In [6]:
docs = [document.text for document in documents]
# iterate over docs and see if the first sentence contains the title, then remove the whole line from the text
all_sentences = []
for i, doc in enumerate(docs):
    sentences = doc.split("\n")
    all_sentences.append(sentences)


In [7]:

all_titles = []
for i, sentences in enumerate(all_sentences):
    page_headings = []
    for j, sentence in enumerate(sentences):
        # sentence is a heading if it starts with # or ## or ### or #### or #####
        if sentence.startswith("#"):
            page_headings.append(sentence)
    all_titles.append(page_headings)

print(len(all_titles))
all_titles[:5]

53


[['# HVAC Service Franchises in the US',
  '## Covid-19 Update',
  '## Industry at a Glance'],
 ['# Key Statistics'],
 ['# Key Industry Data',
  '## Number of Establishments',
  '## Number of Enterprises',
  '## Employment',
  '## Total Wages'],
 ['# Private Spending on Home Improvements',
  '## CAGR:',
  '### 2004 - 2023',
  '### PROJECTED:',
  '#### 2023 - 2029',
  '## Executive Summary'],
 ['## Current Performance',
  '## The Residential Sector Provided a Massive Boost for HVAC Services',
  '## Rising Interest Rates Affected Demand for Both Major Downstream Markets']]

In [8]:
PAGE_TITLE_PROMPT = """ You are expert in determining the page numbers where a specific section begins from the list of titles and corresponding page numbers.
There are 6 sections in the document: ["Industry at a Glance", "Supply Chain", "Competitive Landscape", "Costs & Operations", "Questions for Owners", "Datatables & Glossary"]
Section 1: Industry at a Glance contains subsections such as Key Statistics, Executive Summary, Current Performance, Future Outlook, Industry Definition, Industry Impact, SWOT Analysis, Key Trends.
Section 2: Supply Chain contains subsections such as External drivers, Supply Chain, Similar Industries, Related International Industries, Products & Services, Demand Determinants, Market Segmentation, Business Locations
Section 3: Competitive Landscape contains subsections such as Basis of Competition, Barriers to Entry, Market Share Concentration, Industry Globalization
Section 4: Costs & Operations contains subsections such as Cost Structure, Capital Intensity, Revenue Volatility, Regulation & Policy, Industry Assistance
Section 5: Questions for Owners contains some questions and answers
Section 6: Datatables & Glossary contains some tables for Industry Data and glossary of industry terms.
You will be given a list of titles parsed from pdf and the corresponding page number where each title was parsed from. Each title begins in a new line with a page number as prefix enclosed in angle brackets <>.
Use the given information and your best jusdgement to determine the page number where each section begins.
Use the tool to output the page numbers in json format. If you are not sure about any section, output 0.
"""

In [20]:
from Templates.aws_templates_common import build_aws_template
from Templates.aws_section_page_number_template import TEMPLATE as page_number_template

def get_page_numbers_for_sections(all_titles: List[List[str]]):
    template = page_number_template
    template_parts = template['data']
    page_title_template = build_aws_template(template_parts, tool_name="page_number_inference")[0]

    #add page number as prefix to each title
    all_titles_with_page = []
    for i, titles in enumerate(all_titles):
        titles_with_page = [f"<Page {i+1}> {title}" for title in titles]
        all_titles_with_page.extend(titles_with_page)
    print(all_titles_with_page)
    titles_text = "\n".join(all_titles_with_page)
    num_failed = 0
    response = None
    response_raw = None
    while num_failed < 5:
        try:     
            response, response_raw = response_to_text(titles_text, page_title_template, PAGE_TITLE_PROMPT, None, None, tool_name="page_number_inference")
            break
        except Exception as e:
            print(f"Failed to get page numbers for sections. Retrying in 60 seconds.")
            print(e)
            time.sleep(60)
            num_failed += 1
    return response, response_raw

In [22]:
def get_page_mapping_list(page_mapping, sections_names):
    if page_mapping is None:
        page_mapping = [0]*len(sections_names)
    page_mapping_list = []

    total_pages = get_number_of_pages(PDF_LOCATION)
    for sec_ix, sec in enumerate(sections_names):
        sec_mod = sec.lower().replace(' & ','_').replace(' ', '_')
        f_p = page_mapping[sec_mod]
        prev_ix = sec_ix-1
        while f_p == 0 and prev_ix >= 0:
            prev_sec = sections_names[prev_ix].lower().replace(' & ','_').replace(' ', '_')
            f_p = page_mapping[prev_sec]
            prev_ix -= 1   
        if f_p == 0:
            f_p = 1
        
        if sec_ix == len(sections_names)-1:
            e_p = total_pages
        else:
            sec_next = sections_names[sec_ix+1].lower().replace(' & ','_').replace(' ', '_')
            e_p = min(page_mapping[sec_next], total_pages)
        next_ix = sec_ix+1
        while e_p == 0 and next_ix < len(sections_names):
            next_sec = sections_names[next_ix].lower().replace(' & ','_').replace(' ', '_')
            e_p = page_mapping[next_sec]
            next_ix += 1
        if e_p == 0:
            e_p = total_pages
        page_mapping_list.append((sec, f_p, e_p))
    return page_mapping_list
        

# Phase 3: Extract info from respective section

In [23]:
INFO_EXTRACTION_PROMPT = """
You are an expert in extracting market and financial data from documents.
Use the given tool to extract essential data from text in the enclosed document. Do not make any assumptions or add any information that is not present in the text.

Return the result in JSON format. Do not use non-JSON tags. If some numeric data is not present in the text, simply output the number 101 as an answer where numeric data is expected.
For titles and names, limit the output to 20 words. For descriptions and key points, limit the output to 50 words.
"""

In [30]:
def get_section_text(docs, start_page, end_page):
    section_text = []
    for i in range(start_page-1, end_page):
        section_text.append(docs[i])
    return "\n".join(section_text)
    

In [34]:

def extract_info_for_section(docs, template, main_prompt, start_page, end_page):
    section_text = get_section_text(docs, start_page, end_page)
    num_failed = 0
    result = None
    while num_failed < 5:
        try:
            result = response_to_text(section_text, template, main_prompt, None, None)
            break
        except Exception as e:
            num_failed += 1
            print(f"Error: {e}")
            print(f"Failed {num_failed} times. Sleeping for 60 seconds.")
            time.sleep(60)
            continue
    return result

def extract_info_for_all_sections(page_mapping_list, full_templates, docs):
    section_summaries = []
    raw_responses = []
    for idx, elem in enumerate(page_mapping_list):
        print(f"Extracting for section {elem}")
        section_name, start_page, end_page = elem
        if idx >= len(full_templates):
            break
        section_summary, raw_response = extract_info_for_section(docs, full_templates[idx], INFO_EXTRACTION_PROMPT, start_page, end_page)
        section_summaries.append(section_summary)
        raw_responses.append(raw_response)
        print(f"Extracted for section {section_name}")
        print("Sleeping for 60 seconds.")
        time.sleep(60)
    return section_summaries, raw_responses


In [26]:
sections_names = ["Industry at a Glance", "Supply Chain", "Competitive Landscape", "Costs & Operations", "Questions for Owners", "Datatables & Glossary"]

page_mapping, page_response_raw = get_page_numbers_for_sections(all_titles)


['<Page 1> # HVAC Service Franchises in the US', '<Page 1> ## Covid-19 Update', '<Page 1> ## Industry at a Glance', '<Page 2> # Key Statistics', '<Page 3> # Key Industry Data', '<Page 3> ## Number of Establishments', '<Page 3> ## Number of Enterprises', '<Page 3> ## Employment', '<Page 3> ## Total Wages', '<Page 4> # Private Spending on Home Improvements', '<Page 4> ## CAGR:', '<Page 4> ### 2004 - 2023', '<Page 4> ### PROJECTED:', '<Page 4> #### 2023 - 2029', '<Page 4> ## Executive Summary', '<Page 5> ## Current Performance', '<Page 5> ## The Residential Sector Provided a Massive Boost for HVAC Services', '<Page 5> ## Rising Interest Rates Affected Demand for Both Major Downstream Markets', '<Page 6> # COVID-19 CAUSES DISRUPTIONS IN MOST MARKETS', '<Page 6> # FUTURE OUTLOOK', '<Page 6> ## ELEVATED INTEREST RATES WILL DEPRESS THE RESIDENTIAL SECTOR', '<Page 6> ## A GROWING BUSINESS SECTOR WILL AID HVAC FRANCHISES AS OTHER MARKETS FALTER', '<Page 7> ## ENVIRONMENTALLY FRIENDLY SYSTEMS AR

NameError: name 'get_number_of_pages' is not defined

In [28]:
print(page_mapping)
page_mapping_list = get_page_mapping_list(page_mapping, sections_names)
print(page_mapping_list)

{'industry_at_a_glance': 1, 'supply_chain': 13, 'competitive_landscape': 24, 'costs_operations': 31, 'questions_for_owners': 39, 'datatables_glossary': 43}
[('Industry at a Glance', 1, 13), ('Supply Chain', 13, 24), ('Competitive Landscape', 24, 31), ('Costs & Operations', 31, 39), ('Questions for Owners', 39, 43), ('Datatables & Glossary', 43, 53)]


In [35]:
import pickle
from Templates.ibis_aws_summary_template_all import TEMPLATE as IBIS_SUMMARY_TEMPLATE


template_parts = IBIS_SUMMARY_TEMPLATE['data']
full_templates = build_aws_template(template_parts)

pickle.dump((page_mapping, page_response_raw), open(f"{OUTPUT_FOLDER}/page_mappings.pkl", "wb"))
# print(full_templates)
section_summaries, raw_responses = extract_info_for_all_sections(page_mapping_list, full_templates, docs)

section_results = {"section_summaries": section_summaries, "raw_responses": raw_responses}

pickle.dump(section_results, open(f"{OUTPUT_FOLDER}/section_summaries.pkl", "wb"))

Extracting for section ('Industry at a Glance', 1, 13)
Extracted for section Industry at a Glance
Sleeping for 60 seconds.
Extracting for section ('Supply Chain', 13, 24)
Extracted for section Supply Chain
Sleeping for 60 seconds.
Extracting for section ('Competitive Landscape', 24, 31)
Extracted for section Competitive Landscape
Sleeping for 60 seconds.
Extracting for section ('Costs & Operations', 31, 39)
Extracted for section Costs & Operations
Sleeping for 60 seconds.
Extracting for section ('Questions for Owners', 39, 43)
Extracted for section Questions for Owners
Sleeping for 60 seconds.
Extracting for section ('Datatables & Glossary', 43, 53)


In [39]:
section_summaries[2]

{'barriers_to_entry': {'barriers_level': 'Moderate',
  'barriers_trend': 'Steady',
  'factors_increased_barrier': ['Mature industry life cycle stage',
   'High and increasing competition',
   'State and local regulations limiting operations',
   'Significant start-up costs between $77,000 and $550,000'],
  'factors_decreased_barrier': ['Low capital intensity',
   'Low market concentration',
   'Low globalization'],
  'barriers_points': [{'barrier_title': 'Legal regulations',
    'barrier_description': 'New entrants must comply with different state and local regulations, limiting state-to-state operations.'},
   {'barrier_title': 'Start-up costs',
    'barrier_description': 'Significant start-up costs between $77,000 and $550,000 for new HVAC service entrants.'},
   {'barrier_title': 'Labor intensity',
    'barrier_description': 'HVAC installation requires workers with significant skills and knowledge, intensified by technological progress.'}]},
 'basis_of_competition': {'basis_level': 

In [36]:
from build_markdown_report import build_markdown_report_func

report_md = build_markdown_report_func(section_summaries)

KeyError: 'imports'

In [ ]:
print(report_md)
# save the markdown report
with open(f"{OUTPUT_FOLDER}/summary_report.md", "w") as file:
    file.write(report_md)

In [ ]:
from Templates.build_markdown_report import report_order
import json
json_path = f"{OUTPUT_FOLDER}/section_summaries.json"
with open(json_path, "w") as f:
    json.dump(section_summaries, f, indent=4)

report_order_json_path = f"{OUTPUT_FOLDER}/report_order.json"
with open(report_order_json_path, "w") as f:
    json.dump(report_order, f, indent=4)